# Imports

In [1]:
import pandas as pd

# Clean data

In [57]:
df = pd.read_csv('data/french_apartment.csv')

df.area = df.area.round(2)

# Calculate category

In [58]:
average_price = {
  'Paris': 9224,
  'Lyon': 4717,
  'Marseille': 3680
}

def categorize_price(row):
  price = row['price'] / row['area']
  average = average_price[row.city]

  if price < average * 0.7:
    return 'low'
  elif price < average * 1.3:
    return 'normal'
  elif price < average * 1.7:
    return 'high'
  else:
    return 'scam'
  
df['category'] = df.apply(categorize_price, axis=1)

df.head()

,price,area,years of construction,balcony,garage,rating,city,category
0,718458,96.31,2023,1,0,5,Marseille,scam
1,234271,148.41,2005,1,0,1,Paris,low
2,819375,136.93,2015,0,1,3,Marseille,high
3,841559,170.10,2013,1,0,1,Lyon,normal
4,594298,38.50,2022,1,1,5,Marseille,scam
